In this notebook we will learn how to use the PULSE toolbox as way to study quantum dot experiments... but in Theory

In [1]:
# We are importing all the stuff we need for the program to work
# If we crash here, we are missing some packages and need to install them

import numpy as np
from matplotlib import pyplot as plt
import pyaceqd.pulsegenerator as pg 
import os as os 
# this imports the PULSE modules we will use
from time import sleep
from Pulse_v2 import pulse_shaper_obj, attenuator, motor, spectrometer, power_meter, time_delay, simulator

# this imports the drivers for the hardware we will use
# replace with the actual drivers 
from Pulse_v2 import fake_motor # a closed loop linear motor, e.g. GMT 
from Pulse_v2 import fake_spectrometer # a spectrometer, e.g. Andor 
from Pulse_v2 import fake_power_meter # a power meter, e.g. thorlabs 
from Pulse_v2 import fake_attenuator # an attenuator, e.g. thorlabs 

In the next few cells we are setting up the experimental setup step by step 

In [2]:
# In the lab we probably have a laser source spectrally close to the quantum dot we want to excite
# For now we assume that the exciton emission will be around 780 nm, more on that later 
# Therefore we create an initial pulse object that fits our needs using the pulsegenerator module

initial_pulse = pg.PulseGenerator(t0 = 0, tend=20, dt=0.2, central_wavelength=780)
#the argument central_wavelength should be close to the exciton emission wavelength
#t0, tend and dt specivy the time range and resolution of the pulse, directly related to the spectral resolution in energy domain. If problems like low spectral resolution, or limited spectral range arise, these parameters should be adjusted. (increase tend - t0 for better resolution, decrease dt for higher range)

initial_pulse.add_gaussian_time(unit='nm', width_t=0.230, central_f=781.5, sig_or_fwhm='fwhm', t0=10) 
# whe create a gaussian pulse with a FWHM of 130 fs, centered at 781.5 nm, starting at 10 ps on our time axis. This should be close to what the Chamelion should be able to do. You can also load pulses from measured spectra, or create other shapes.  

# We can define a power of the initial pulse, depending of the setup this might vary much.
# for fs pulses values of ~1000 have prooven to be a good starting point
initial_pulse.set_pulse_power(1000)



In [3]:
# We will first setup a pulse shaper setup consisting of a pulse shaper, an attenuator and a spectrometer as measuring device 

# We (might) need to create fake objects for certain devices to set some parameters 
lab_motor = fake_motor()
lab_attenuator = fake_attenuator()
lab_spectrometer = fake_spectrometer(start_wl=770, end_wl=790, n_wl=1340) 
# using the fake_spectrometer class we define the window we will be able to see on the spectrometer. Although it might seem not needed for simulation purposes, the point of this toolbox is to predict experimental observations.

# We than create the PULSE objects from the devices we have
# The attenuator and the spectrometer can already be initialized as: 
attenuator_A = attenuator(device=lab_attenuator, name='attenuator_A')
spectrometer_A = spectrometer(device=lab_spectrometer, name='spectrometer_A')

# The pulse shaper needs a calibration file that is usually created by running "pulse_shaper_calibration.py", see the notebook DEMO_pulse_Experiment.ipynb 
# In short such a file contains information about the relationship of the motor position and the spectral position of the pulse shaper. Additionally it contains the spectral width and sharpness of the slit used in the pulse shaper.
#For now we will use the file below, which is was callibrated before.

pulse_shaper_calibration = 'calibration_slit_13.txt'
pulse_motor = motor(device=lab_motor, name='pulse_motor')
pulse_shaper_A = pulse_shaper_obj(device=pulse_motor, calibration_file=pulse_shaper_calibration, name='pulse_shaper_A')


Attenuator connected!
Attenuator name: attenuator_A
Attenuator device: <Pulse_v2.fake_attenuator object at 0x7f14b83eb610>
Attenuator time of creation: 2024-10-01 15:08:02.680233

Spectrometer connected!
Spectrometer name: spectrometer_A
Spectrometer device: <Pulse_v2.fake_spectrometer object at 0x7f14b83eaec0>
Spectrometer time of creation: 2024-10-01 15:08:02.680470

Motor connected!
Motor name: pulse_motor
Motor device: <Pulse_v2.fake_motor object at 0x7f14b83eaef0>
Motor time of creation: 2024-10-01 15:08:02.681496

Pulse Shaper connected!
Pulse Shaper name: pulse_shaper_A
Pulse Shaper device: <Pulse_v2.motor object at 0x7f1468ab5630>
Pulse Shaper time of creation: 2024-10-01 15:08:02.681630
Pulse Shaper calibration file: calibration_slit_13.txt


We don't really need to check the drivers for a simulation and there is no need to check through all the devices

In [ ]:
# We will now open the control class of each device and chain them together 

# The first controller wil be the pulse shaper itself and gets as input the initial pulse object
ps_controller = pulse_shaper_A.open_control(pulse_object=initial_pulse)
# the second one will be the attenuator, which gets as input the pulse shaper controller 
att_controller = attenuator_A.open_control(previous_control=ps_controller)
# This tells the attenuator to act on the pulse leaving the pulse shaper 
spec_controller = spectrometer_A.open_control(previous_control=att_controller) 
# A measurement device measures certain properties of the pulse

# if we want a GUI, and we do, we must start the mainloow from any of the controllers

#ps_controller.start_gui() # <---- this is the mainloop

#att_controller.start_gui() # <---- or this
#spec_controller.start_gui() # <---- or this 

In [ ]:
ps_controller = pulse_shaper_A.open_control(pulse_object=initial_pulse)

att_controller = attenuator_A.open_control(previous_control=ps_controller)

spec_controller = spectrometer_A.open_control(previous_control=att_controller) 

# if we want to measure multiple things we just add another measurement device to the chain
lab_power_meter = fake_power_meter()
power_meter_A = power_meter(device=lab_power_meter, name='power_meter_A')
power_controller = power_meter_A.open_control(previous_control=att_controller)
##########
#ps_controller.start_gui() # <<<<<<<<<

Now we want to setup a simulator and simulate photon emission and dynamics of a quantum dot 

In [4]:
# First we need to have a quantum dot in mind and generate calibration file for it. You can also just edit the txt file if needed.
import configparser
calibration_name_qd = 'DEMO_calibration_QD.txt'

config = configparser.ConfigParser()
        
# emsiion wavelength in nm (don't worry about what dark says)
config['EMISSION'] = {
    'exciton_wavelength':str(779.89),
    'biexciton_wavelength':str(781.79),
    'dark_wavelength':str(779.81)}

#splitting in mueV
config['SPLITTING'] = {
    'fss_bright':str(0),
    'fss_dark':str(0)}

# lifetimes in ps
config['LIFETIMES'] = {
    'exciton':str(180),
    'biexciton':str(90)}
# g_factors (don't worry)
config['G_FACTORS'] = {'g_ex':'0', 'g_ez':'0', 'g_hx':'0', 'g_hz':'0'}

with open(calibration_name_qd, 'w') as configfile:
    config.write(configfile)


In [5]:
# Now we can initialize the simulator
sim = simulator(qd_calibration= calibration_name_qd, name='simulator',sim_kind='ace')
# if ace is installed, you can use the sim_kind = "ace" for a 4 level system or "ace_6ls" for a 6 level system. Else you can put "qutip" for a four level system.


Simulator connected!
Quantum dot calibration file: DEMO_calibration_QD.txt
Simulation method: ace
Temp directory: 


In [ ]:
ps_controller = pulse_shaper_A.open_control(pulse_object=initial_pulse)

att_controller = attenuator_A.open_control(previous_control=ps_controller)

sim_controller = sim.open_control(previous_control=att_controller)
spec_controller = spectrometer_A.open_control(previous_control=att_controller, simulation_control=sim_controller)
# Note that the spec controller now also gets the simulation controller as input
# the power meter does not  

#power_controller = power_meter_A.open_control(previous_control=att_controller)

# the power meter causes a slow response...maybe it updates to much, don't know yet

##########
ps_controller.start_gui() # <<<<<



In [6]:
# Now we can build more complex systems. 
# As an example we will control a Stim-TPE experiment, adding another set of pulse shaper, attenuator and a time delay stage 

# we add a different pulse shaper, motor and calibration file to the system
lab_motor_B = fake_motor()
pulse_motor_B = motor(device=lab_motor_B, name='pulse_motor_B')

pulse_shaper_calibration_B = 'calibration_slit_18.txt'
pulse_shaper_B = pulse_shaper_obj(device=pulse_motor_B, calibration_file=pulse_shaper_calibration_B, name='pulse_shaper_B')

lab_attenuator_B = fake_attenuator()
attenuator_B = attenuator(device=lab_attenuator_B, name='attenuator_B')

lab_motor_C = fake_motor()
pulse_motor_C = motor(device=lab_motor_C, name='pulse_motor_C')
delay_stage = time_delay(device=pulse_motor_C, name='delay_stage',num_passes=2, offset=30)
# The delay stage also takes a motor as input. The control_value is excpected in ps and will be handed to the motor in mm. You can add e.g. unit = 10 to go from mm to cm 

# in order to see the stim tpe we will need a new pulse with higher temporal duration 
initial_pulse_2 = pg.PulseGenerator(t0 = 0, tend=60, dt=0.2, central_wavelength=780)
initial_pulse_2.add_gaussian_time(unit='nm', width_t=0.230, central_f=781.5, sig_or_fwhm='fwhm', t0=10)
initial_pulse_2.set_pulse_power(1000)




Motor connected!
Motor name: pulse_motor_B
Motor device: <Pulse_v2.fake_motor object at 0x7f1468ab5f30>
Motor time of creation: 2024-10-01 15:08:15.474751

Pulse Shaper connected!
Pulse Shaper name: pulse_shaper_B
Pulse Shaper device: <Pulse_v2.motor object at 0x7f1468ab5810>
Pulse Shaper time of creation: 2024-10-01 15:08:15.475044
Pulse Shaper calibration file: calibration_slit_18.txt

Attenuator connected!
Attenuator name: attenuator_B
Attenuator device: <Pulse_v2.fake_attenuator object at 0x7f1468ab58d0>
Attenuator time of creation: 2024-10-01 15:08:15.477086

Motor connected!
Motor name: pulse_motor_C
Motor device: <Pulse_v2.fake_motor object at 0x7f1468ab7df0>
Motor time of creation: 2024-10-01 15:08:15.477180

Time Delay connected!
Time Delay name: delay_stage
Time Delay device: <Pulse_v2.motor object at 0x7f1468ab5c30>
Time Delay time of creation: 2024-10-01 15:08:15.477257


In [ ]:

# first pulse shaper unit 
ps_controller_A = pulse_shaper_A.open_control(pulse_object=initial_pulse_2)
att_controller_A = attenuator_A.open_control(previous_control=ps_controller_A)
delay_controller_A = delay_stage.open_control(previous_control=att_controller_A)
# the first pulse we also send through the delay stage 

# second pulse shaper unit
ps_controller_B = pulse_shaper_B.open_control(pulse_object=initial_pulse_2)
att_controller_B = attenuator_B.open_control(previous_control=ps_controller_B)

sim_controller_B = sim.open_control(previous_control=[att_controller_B,delay_controller_A])
# if two or more controllers are given the pulses will be combined for the simulation 
# The same for the spectrometer... A polarisation measurement before the spectrometer will be added in the future
spec_controller_B = spectrometer_A.open_control(previous_control=[att_controller_B,delay_controller_A], simulation_control=sim_controller_B)

# again open the GUI
spec_controller_B.start_gui() # <<<<<<<


 

In [7]:
# See how nice that works? Finding the right parameters was quite tedious though 
# this is why we want to import the control_optimizer module, which will help us to find the right parameters for an optimum measurement task

# since the measurement is yet lacking a polarisation measurement, we will use a high fss qd for ilustrating the situation better ... 
import configparser
calibration_name_qd = 'DEMO_calibration_QD_high_fss.txt'

config = configparser.ConfigParser()
        
# emsiion wavelength in nm (don't worry about what dark says)
config['EMISSION'] = {
    'exciton_wavelength':str(779.89),
    'biexciton_wavelength':str(781.79),
    'dark_wavelength':str(779.81)}

#splitting in mueV
config['SPLITTING'] = {
    'fss_bright':str(200), # <---- this is the important part
    'fss_dark':str(0)}

# lifetimes in ps
config['LIFETIMES'] = {
    'exciton':str(180),
    'biexciton':str(90)}
# g_factors (don't worry)
config['G_FACTORS'] = {'g_ex':'0', 'g_ez':'0', 'g_hx':'0', 'g_hz':'0'}

with open(calibration_name_qd, 'w') as configfile:
    config.write(configfile)

In [8]:
# we need to change the simulator to the high fss qd
sim.set_qd_calibration(calibration_name_qd)

# then open all the controllers again
# first pulse shaper unit 
ps_controller_A = pulse_shaper_A.open_control(pulse_object=initial_pulse_2)
att_controller_A = attenuator_A.open_control(previous_control=ps_controller_A)
delay_controller_A = delay_stage.open_control(previous_control=att_controller_A)
# the first pulse we also send through the delay stage 

# second pulse shaper unit
ps_controller_B = pulse_shaper_B.open_control(pulse_object=initial_pulse_2)
att_controller_B = attenuator_B.open_control(previous_control=ps_controller_B)

sim_controller_B = sim.open_control(previous_control=[delay_controller_A,att_controller_B])
# if two or more controllers are given the pulses will be combined for the simulation 
# The same for the spectrometer... A polarisation measurement before the spectrometer will be added in the future
spec_controller_B = spectrometer_A.open_control(previous_control=[delay_controller_A,att_controller_B], simulation_control=sim_controller_B, open_gui=False)
# To make things harder for us we will supress the gui of the spectrometer and modify a few things

spec_controller_B.set_simulation_counts(750) # maximum emission (theory gives 1) is 750 + background counts  
spec_controller_B.set_simulation_background(45) # we have a spectrometer background of 45 counts 
spec_controller_B.set_simulation_gaussian_noise(8) # on top we add gaussian noise of std. dev. 8 counts 
spec_controller_B.set_pulse_scale(0.5) # the pulse's visibility is reduced. The optimizer currently "sees" the sum of emission lines and pulse object. This will be better when polarisation measurements are implemented. For now it can be used to "cross pol" if set rather low. 

# also while we're at it we will turn the thing pn and press some buttons 
spec_controller_B.toggle_display_experiment()
spec_controller_B.toggle_display_simulation()
spec_controller_B.toggle_display_pulse()
spec_controller_B.toggle_running()

spec_controller_B.gui()
# and then call the gui() funtion

### import the package
import control_optimizer as co
# and create the optimizer object. device:control is a list of all the controllers that should be optimized, measururement_control is the measurment device that is optimized on. The stiring measurment_kind specifies the kind of measurement that is done. This for sure can be done differently and better, but for now "spectrometer" optimizes of the max, mean or sum of a given collection window.

# it seems like the order in device_control is important. The first one being one that gets the "initial_pulse" object seems to work. 
optimizer = co.control_optimizer(device_control=[ps_controller_A,att_controller_A,delay_controller_A,ps_controller_B,att_controller_B],measururement_control=[spec_controller_B],measurement_kind='spectrometer',open_gui=True)

#optimizer.gui()
# again open the GUI
optimizer.start_gui() # <<<<<<<

#spec_controller_B.start_gui() # <<<<<<<


Simulator connected!
Quantum dot calibration file: DEMO_calibration_QD_high_fss.txt
Simulation method: ace
Temp directory: 
Moving to position:  14.799999999999924
Pulseshaper: pulse_shaper_A
Current position: 781.7152628346472
Current step size: 0.1
Setting attenuation to:  0.5
Current attenuation:  0.5
Step size:  0.02
Large step size:  10
Initial attenuation:  None
Open GUI:  True
Parent window:  None
Previous control:  <pulse_shaper_control.pulse_shaper_control object at 0x7f1468ab79d0>
Moving to position:  -4.49688
Current delay:  30.0
Step size:  1.0
Large step size:  10
Initial delay:  None
Open GUI:  True
Parent window:  None
Previous control:  <attenuator_control.attenuator_control object at 0x7f1468ab5690>
Moving to position:  17.09999999999995
Pulseshaper: pulse_shaper_B
Current position: 780.2162259787176
Current step size: 0.1
Setting attenuation to:  0.5
Current attenuation:  0.5
Step size:  0.02
Large step size:  10
Initial attenuation:  None
Open GUI:  True
Parent wind

/home/florian/Leaving version PULSE/PulseGenerationACE/PULSE/simulator_control.py:176: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  self.ax_pulses.set_ylim([0,1.1*np.max([np.abs(self.pulse_object.temporal_representation_x),np.abs(self.pulse_object.temporal_representation_y)])])


Moving to position:  16.18113221088901
Setting attenuation to:  0.3361089524248809
Output:  397.5297837647995
Moving to position:  16.181089799575037
Setting attenuation to:  0.35163621596598255
Output:  391.36863017825954
Moving to position:  16.167830996947057
Setting attenuation to:  0.3340401110019465
Output:  407.7959438648945
Moving to position:  16.149575865931457
Setting attenuation to:  0.3326005791612598
Output:  413.7137515806712
Moving to position:  16.108341058944465
Setting attenuation to:  0.3536656167012979
Output:  389.82856649056316
Moving to position:  16.147025145734265
Setting attenuation to:  0.32843118133879723
Output:  416.212664178064
Moving to position:  16.144567265323847
Setting attenuation to:  0.31880893495175133
Output:  413.5429907379082
Moving to position:  16.14590306670998
Setting attenuation to:  0.33600482334667575
Output:  413.23932055122674
Moving to position:  16.147025145734265
Setting attenuation to:  0.32843118133879723
Output:  412.4408944170

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/usr/lib/python3.10/tkinter/__init__.py", line 839, in callit
    func(*args)
  File "/home/florian/Leaving version PULSE/PulseGenerationACE/PULSE/spectrometer_control.py", line 244, in update_gui
    self.update_measurement()
  File "/home/florian/Leaving version PULSE/PulseGenerationACE/PULSE/spectrometer_control.py", line 186, in update_measurement
    self.update_previous_control()
  File "/home/florian/Leaving version PULSE/PulseGenerationACE/PULSE/spectrometer_control.py", line 139, in update_previous_control
    self.previous_control[j].update_gui()
  File "/home/florian/Leaving version PULSE/PulseGenerationACE/PULSE/attenuator_control.py", line 185, in update_gui
    self.set_step_size(float(self.step_size_entry.get()))
  File "/usr/lib/python3.10/tkinter/__init__.py", line 3082, in get
    return self.tk.c

Motor closed
delay_stage closed
Motor closed
pulse_shaper_A closed
